In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

from itertools import combinations
np.random.seed(7)
from scipy.optimize import fmin_tnc
from numpy.linalg import inv
import random
import time

In [ ]:
## From Oh & Iyengar (non-public code)
class mnlEnv:
    def __init__(self, theta, K):
        super(mnlEnv, self).__init__()
        self.theta = theta
        self.K = K
        
    def compute_rwd(self, means):
        u = np.exp(means)
        uSum = 1 + u.sum()
        prob = np.append(u, [1])/uSum
        rwd = u.sum()/uSum
        Y = np.random.multinomial(1, prob)
        return rwd, Y
    
    def get_opt_rwd(self, x):
        opt_means = np.sort(np.dot(x,self.theta))[::-1][:self.K]
        rwd, Y = self.compute_rwd(opt_means)
        return rwd


In [ ]:
## From Oh & Iyengar (non-public code)
class RegularizedMNLRegression:

    def compute_prob(self, theta, x):
        means = np.dot(x, theta)
        u = np.exp(means)
        u_ones = np.column_stack((u,np.ones(u.shape[0])))
        logSumExp = u_ones.sum(axis=1)
        prob = u_ones/logSumExp[:,None]
        return prob

    def cost_function(self, theta, x, y, lam):
        m = x.shape[0]
        prob = self.compute_prob(theta, x)
        return -(1/m)*np.sum( np.multiply(y, np.log(prob))) + (1/m)*lam*np.linalg.norm(theta)

    def gradient(self, theta, x, y, lam):
        m = x.shape[0]
        prob = self.compute_prob(theta, x)
        eps = (prob-y)[:,:-1]
        grad = (1/m)*np.tensordot(eps,x,axes=([1,0],[1,0])) + (1/m)*lam*theta
        return grad

    def fit(self, x, y, theta, lam):
        opt_weights = fmin_tnc(func=self.cost_function, x0=theta, fprime=self.gradient, args=(x, y, lam))
        self.w = opt_weights[0]
        return self

In [ ]:
## From Oh & Iyengar (non-public code)
class MNLRegression:

    def compute_prob(self, theta, x, y):
        means = np.dot(x, theta)
        u = np.exp(means)
        u_ones = np.column_stack((u,np.ones(u.shape[0])))
        logSumExp = u_ones.sum(axis=1)
        prob = u_ones/logSumExp[:,None]
        return prob

    def cost_function(self, theta, x, y):
        m = x.shape[0]
        prob = self.compute_prob(theta, x, y)
        return -(1/m)*np.sum( np.multiply(y, np.log(prob)))

    def gradient(self, theta, x, y):
        m = x.shape[0]
        prob = self.compute_prob(theta, x, y)
        eps = (prob-y)[:,:-1]
        grad = (1/m)*np.tensordot(eps,x,axes=([1,0],[1,0]))
        return grad

    def fit(self, x, y, theta):
        opt_weights = fmin_tnc(func=self.cost_function, x0=theta, fprime=self.gradient, args=(x, y))
        self.w = opt_weights[0]
        return self

In [ ]:
## From Oh & Iyengar (non-public code)
class ts_mnl:
    def __init__(self, N, K, d, T0, kappa = 0.5, alpha=None):
        super(ts_mnl, self).__init__()
        self.N=N
        self.K=K
        self.d=d
        self.X=np.zeros((K,d))[np.newaxis, ...]
        self.Y=np.zeros(K+1)[np.newaxis, ...]
        self.T0=T0
        self.kappa=kappa
        self.theta=np.zeros(d)
        self.V=np.zeros((d, d))
        self.mnl=MNLRegression()
        self.alpha=alpha
        
    def choose_S(self,t,x):  # x is N*d matrix
        if t < self.T0:
            self.S = np.random.choice(self.N,self.K, replace=False)
        else:
            if self.alpha is None:
                self.alpha = (1/(2*self.kappa))*np.sqrt(2*self.d*np.log(1+t/self.d)+2*np.log(t))
            theta_tilde = np.random.multivariate_normal(self.theta, np.square(self.alpha)*inv(self.V))
            means = np.dot(x,theta_tilde)            
            self.S = np.argsort(means)[::-1][:self.K]
        self.X = np.concatenate((self.X, x[self.S,:][np.newaxis, ...]))
        self.V += np.matmul(x[self.S,:].T, x[self.S,:])
        return(self.S)

    def update_theta(self,Y,t):
        self.Y = np.concatenate((self.Y, Y[np.newaxis, ...]))
        if t==2:
            self.X = np.delete(self.X, (0), axis=0)
            self.Y = np.delete(self.Y, (0), axis=0)
        if t>self.T0:
            self.mnl.fit(self.X, self.Y, self.theta)
            self.theta = self.mnl.w


In [ ]:
## From Oh & Iyengar (non-public code)
#UCB-MNL
class ucb_mnl:
    def __init__(self, N, K, d, T0, kappa = 10000, alpha=None):
        super(ucb_mnl, self).__init__()
        self.N=N
        self.K=K
        self.d=d
        self.X=np.zeros((K,d))[np.newaxis, ...]
        self.Y=np.zeros(K+1)[np.newaxis, ...]
        self.T0=T0
        self.kappa=kappa
        self.theta=np.zeros(d)
        self.V=np.zeros((d, d))
        self.mnl = MNLRegression()
        self.alpha=alpha
        self.prev_S = np.zeros(K)
        
    def choose_S(self,t,x):  # x is N*d matrix
        if t < self.T0:
            self.S = np.random.choice(self.N,self.K, replace=False)
        else:
            if self.alpha is None:
                self.alpha = (1/(2*self.kappa))*np.sqrt(2*self.d*np.log(1+t/self.d)+2*np.log(t))
            means = np.dot(x,self.theta)
            xv = np.sqrt((np.matmul(x, inv(self.V)) * x).sum(axis = 1))
            u = means + self.alpha * xv
            self.S = np.argsort(u)[::-1][:self.K]
        self.X = np.concatenate((self.X, x[self.S,:][np.newaxis, ...]))
        self.V += np.matmul(x[self.S,:].T, x[self.S,:])
        return(self.S)

    def update_theta(self,Y,t):
        self.Y = np.concatenate((self.Y, Y[np.newaxis, ...]))
        if t==2:
            self.X = np.delete(self.X, (0), axis=0)
            self.Y = np.delete(self.Y, (0), axis=0)
        if t>self.T0:
            self.mnl.fit(self.X, self.Y, self.theta)
            self.theta = self.mnl.w




In [ ]:
#CB-MNL
class cb_mnl2:
    def __init__(self, N, K, d, lbd, env_theta):
        super(cb_mnl2, self).__init__()
        self.N=N
        self.K=K
        self.d=d
        self.env_theta = env_theta
        self.X=np.zeros((K,d))[np.newaxis, ...]
        self.Y=np.zeros(K+1)[np.newaxis, ...]
        self.theta=np.ones(d)/(np.linalg.norm(np.ones(d)))
        #self.V=np.zeros((d, d))
        self.mnl = RegularizedMNLRegression()
        self.lbd = lbd
        #self.prev_S = np.zeros(K)
        self.S = np.zeros(K)
        self.max_assort_theta = np.ones(d)/(np.linalg.norm(np.ones(d)))
        
    def choose_S(self,t,x):  # x is N*d matrix
        # generate theta in lattice
        if self.d == 1:
          theta_base_set = np.mgrid[-1:1:.2]
        elif self.d == 2:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 3:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 4:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 5:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]

        total_theta = round((theta_base_set.size)/self.d)
        ran = round((total_theta)**(1/self.d))

        beta_sq = .0001*self.d*np.log(self.K*t)
        max_assort_val= 0
        max_assort_single = []
        max_assort_theta =[]
        counter_theta = counter_unitball=0
        for i in range(0,ran):
          for j in range(0,ran):
            for k in range(0,ran):
              theta_val = theta_base_set[:,i,j,k]
              if np.linalg.norm(theta_val) <= 1:
                counter_unitball = counter_unitball+1
                if self.in_confidence_set(theta_val,beta_sq)==1:
                  counter_theta = counter_theta +1
                  means = np.dot(x,theta_val)
                  assort_single = np.argsort(means)[::-1][:self.K]
                  assort_means = np.dot(x[assort_single,:],theta_val)
                  assort_val = self.assort_value(assort_means)
                  if assort_val > max_assort_val:
                    max_assort_single = assort_single
                    max_assort_val = assort_val
                    self.max_assort_theta = theta_val
        
        if len(max_assort_single) ==0:
          means = np.dot(x,theta_val)
          assort_single = np.argsort(means)[::-1][:self.K]
          self.S = assort_single
          print("time", t) 
        else:
          self.S =max_assort_single
        self.X = np.concatenate((self.X, x[self.S,:][np.newaxis, ...]))

        return(self.S)
        

    def update_theta(self,Y,t):
        self.Y = np.concatenate((self.Y, Y[np.newaxis, ...]))
        if t==2:
            self.X = np.delete(self.X, (0), axis=0)
            self.Y = np.delete(self.Y, (0), axis=0)
        self.lbd = 2*d*np.log(self.K*t)
        self.mnl.fit(self.X, self.Y, self.theta, self.lbd)
        self.theta = self.mnl.w
    
    
    def in_confidence_set(self,theta_val,beta_sq):
        
        log_loss_theta_val = self.cost_function(theta_val,self.X,self.Y,lbd)
        log_loss_theta_est = self.cost_function(self.theta,self.X,self.Y,lbd)
        if  log_loss_theta_est < beta_sq + log_loss_theta_val:
          return 1
        else:
          return 0

    def compute_prob(self, theta, x):
        means = np.dot(x, theta)
        u = np.exp(means)
        u_ones = np.column_stack((u,np.ones(u.shape[0])))
        logSumExp = u_ones.sum(axis=1)
        prob = u_ones/logSumExp[:,None]
        return prob

    def cost_function(self, theta, x, y, lam):
        m = x.shape[0]
        prob = self.compute_prob(theta, x)
        return np.sum( np.multiply(y, np.log(prob))) - .5*lam*np.linalg.norm(theta)

    def assort_value(self,means):
        u = np.exp(means)
        uSum = 1 + u.sum()
        rwd = u.sum()/uSum
        return rwd


In [ ]:
#CB-MNL
class cb_mnl_d5:
    def __init__(self, N, K, d, lbd, env_theta):
        super(cb_mnl_d5, self).__init__()
        self.N=N
        self.K=K
        self.d=d
        self.env_theta = env_theta
        self.X=np.zeros((K,d))[np.newaxis, ...]
        self.Y=np.zeros(K+1)[np.newaxis, ...]
        self.theta=np.ones(d)/(np.linalg.norm(np.ones(d)))
        self.mnl = RegularizedMNLRegression()
        self.lbd = lbd
        self.S = np.zeros(K)
        self.max_assort_theta = np.ones(d)/(np.linalg.norm(np.ones(d)))

    
        
    def choose_S(self,t,x):  # x is N*d matrix
        # generate theta in lattice
        if self.d == 1:
          theta_base_set = np.mgrid[-1:1:.2]
        elif self.d == 2:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 3:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 4:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]
        elif self.d == 5:
          theta_base_set = np.mgrid[-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2,-1:1:.2]

        total_theta = round((theta_base_set.size)/self.d)
        ran = round((total_theta)**(1/self.d))

        beta_sq = .0001*self.d*np.log(self.K*t)
        max_assort_val= 0
        max_assort_single = []
        max_assort_theta =[]
        counter_theta = counter_unitball=0
        for i in range(0,ran):
          for j in range(0,ran):
            for k in range(0,ran):
              for l in range(0,ran):
                for m in range(0,ran):
                  theta_val = theta_base_set[:,i,j,k,l,m]
                  if np.linalg.norm(theta_val) <= 1:
                    counter_unitball = counter_unitball+1
                    ### test in confidence set
                    if self.in_confidence_set(theta_val,beta_sq)==1:
                      counter_theta = counter_theta +1
                      means = np.dot(x,theta_val)
                      assort_single = np.argsort(means)[::-1][:self.K]
                      assort_means = np.dot(x[assort_single,:],theta_val)
                      assort_val = self.assort_value(assort_means)
                      if assort_val > max_assort_val:
                        max_assort_single = assort_single
                        max_assort_val = assort_val
                        self.max_assort_theta = theta_val
        
        if len(max_assort_single) ==0:
          means = np.dot(x,theta_val)
          assort_single = np.argsort(means)[::-1][:self.K]
          self.S = assort_single
        else:
          self.S =max_assort_single
        self.X = np.concatenate((self.X, x[self.S,:][np.newaxis, ...]))

        return(self.S)
        

    def update_theta(self,Y,t):
        self.Y = np.concatenate((self.Y, Y[np.newaxis, ...]))
        if t==2:
            self.X = np.delete(self.X, (0), axis=0)
            self.Y = np.delete(self.Y, (0), axis=0)
        self.lbd = 2*d*np.log(self.K*t)
        self.mnl.fit(self.X, self.Y, self.theta, self.lbd)
        self.theta = self.mnl.w
    
    
    def in_confidence_set(self,theta_val,beta_sq):
        
        log_loss_theta_val = self.cost_function(theta_val,self.X,self.Y,lbd)
        log_loss_theta_est = self.cost_function(self.theta,self.X,self.Y,lbd)
        if  log_loss_theta_est < beta_sq + log_loss_theta_val:
          return 1
        else:
          return 0

    def compute_prob(self, theta, x):
        means = np.dot(x, theta)
        u = np.exp(means)
        u_ones = np.column_stack((u,np.ones(u.shape[0])))
        logSumExp = u_ones.sum(axis=1)
        prob = u_ones/logSumExp[:,None]
        return prob

    def cost_function(self, theta, x, y, lam):
        m = x.shape[0]
        prob = self.compute_prob(theta, x)
        return np.sum( np.multiply(y, np.log(prob))) - .5*lam*np.linalg.norm(theta)

    def assort_value(self,means):
        u = np.exp(means)
        uSum = 1 + u.sum()
        rwd = u.sum()/uSum
        return rwd


In [ ]:

def sample_spherical(N, k):
    vec = np.random.randn(k, N)
    vec /= np.linalg.norm(vec, axis=0)
    return vec

def sample_elliptical(N, d, k, mu):
    S = sample_spherical(N, k)
    A = np.random.rand(d,k)
    R = np.random.normal(size=N)
    return mu + A.dot(R*S)




N = 15
K = 4
d = 5
T = 100
dist = 0

MC_counts = 25
cumulated_regret_cb_mnl2 = np.zeros((MC_counts,T))
cumulated_regret_ucb_mnl = np.zeros((MC_counts,T))
cumulated_regret_ts_mnl = np.zeros((MC_counts,T))
cumulated_regret_ucb_mnl_klow = np.zeros((MC_counts,T))
cumulated_regret_ts_mnl_klow = np.zeros((MC_counts,T))
cumulated_regret_ucb_mnl_kinter = np.zeros((MC_counts,T))
cumulated_regret_ts_mnl_kinter = np.zeros((MC_counts,T))




T0 = 5*np.sqrt(d)
for mc in range(MC_counts):
  cumulated_regret_single_cb_mnl2 =[]
  cumulated_regret_single_ucb_mnl =[]
  cumulated_regret_single_ts_mnl =[]
  cumulated_regret_single_ucb_mnl_klow =[]
  cumulated_regret_single_ts_mnl_klow =[]
  cumulated_regret_single_ucb_mnl_kinter =[]
  cumulated_regret_single_ts_mnl_kinter =[]
  sigma_sq=1
  rho_sq= 0
  W=(sigma_sq-rho_sq)*np.eye(N) + rho_sq*np.ones((N,N))
  dist=0
  theta=np.random.uniform(0.,1.,d)
  env=mnlEnv(theta, K)
  env_theta= theta
  lbd=.001*d*np.log(K*T)
  M1=cb_mnl_d5(N=N, K=K, d=d, lbd=lbd, env_theta=env_theta)
  M2=ucb_mnl(N=N, K=K, d=d, T0=T0, kappa = .001)
  M3=ts_mnl(N=N, K=K, d=d, T0=T0, kappa = .001)
  M4=ucb_mnl(N=N, K=K, d=d, T0=T0, kappa = .1)
  M5=ts_mnl(N=N, K=K, d=d, T0=T0, kappa = .1)
  M6=ucb_mnl(N=N, K=K, d=d, T0=T0, kappa = .01)
  M7=ts_mnl(N=N, K=K, d=d, T0=T0, kappa = .01)
  RWD1=list()
  optRWD=list()
  RWD2=list()
  optRWD2=list()
  RWD3=list()
  optRWD3=list()
  RWD4=list()
  optRWD4=list()
  RWD5=list()
  optRWD5=list()
  RWD6=list()
  optRWD6=list()
  RWD7=list()
  optRWD7=list()
  for t in range(T):
    lbd=.001*d*np.log(K*T)
    if dist == 0:
      x=np.random.multivariate_normal(np.zeros(N),W,d).T
    elif dist == 1:
      x=(np.random.random((N, d)) * 2 - 1)
    elif dist == 2:
      x=sample_elliptical(N, d, int(d/2), 0).T
	
    S1=M1.choose_S(t+1,x)
    rwd1, Y1 = env.compute_rwd(np.dot(x[S1,:],theta))
    RWD1.append(rwd1)
    M1.update_theta(Y1,t+1)
    opt_rwd = env.get_opt_rwd(x)
    optRWD.append(opt_rwd)
    cumulated_regret_single_cb_mnl2.append(np.cumsum(optRWD)-np.cumsum(RWD1))
    
    S2=M2.choose_S(t+1,x)
    rwd2, Y2 = env.compute_rwd(np.dot(x[S2,:],theta))
    RWD2.append(rwd2)
    M2.update_theta(Y2,t+1)
    opt_rwd2 = env.get_opt_rwd(x)
    optRWD2.append(opt_rwd2)
    cumulated_regret_single_ucb_mnl.append(np.cumsum(optRWD2)-np.cumsum(RWD2))

    S3=M3.choose_S(t+1,x)
    rwd3, Y3 = env.compute_rwd(np.dot(x[S3,:],theta))
    RWD3.append(rwd3)
    M3.update_theta(Y3,t+1)
    opt_rwd3 = env.get_opt_rwd(x)
    optRWD3.append(opt_rwd3)
    cumulated_regret_single_ts_mnl.append(np.cumsum(optRWD3)-np.cumsum(RWD3))
  
    S4=M4.choose_S(t+1,x)
    rwd4, Y4 = env.compute_rwd(np.dot(x[S4,:],theta))
    RWD4.append(rwd4)
    M4.update_theta(Y4,t+1)
    opt_rwd4 = env.get_opt_rwd(x)
    optRWD4.append(opt_rwd4)
    cumulated_regret_single_ucb_mnl_klow.append(np.cumsum(optRWD4)-np.cumsum(RWD4))
  
    S5=M5.choose_S(t+1,x)
    rwd5, Y5 = env.compute_rwd(np.dot(x[S5,:],theta))
    RWD5.append(rwd5)
    M5.update_theta(Y5,t+1)
    opt_rwd5 = env.get_opt_rwd(x)
    optRWD5.append(opt_rwd5)
    cumulated_regret_single_ts_mnl_klow.append(np.cumsum(optRWD5)-np.cumsum(RWD5))


    S6=M6.choose_S(t+1,x)
    rwd6, Y6 = env.compute_rwd(np.dot(x[S6,:],theta))
    RWD6.append(rwd6)
    M6.update_theta(Y6,t+1)
    opt_rwd6 = env.get_opt_rwd(x)
    optRWD6.append(opt_rwd6)
    cumulated_regret_single_ucb_mnl_kinter.append(np.cumsum(optRWD6)-np.cumsum(RWD6))
  
    S7=M7.choose_S(t+1,x)
    rwd7, Y7 = env.compute_rwd(np.dot(x[S7,:],theta))
    RWD7.append(rwd7)
    M7.update_theta(Y7,t+1)
    opt_rwd7 = env.get_opt_rwd(x)
    optRWD7.append(opt_rwd7)
    cumulated_regret_single_ts_mnl_kinter.append(np.cumsum(optRWD5)-np.cumsum(RWD5))
  cumulated_regret_cb_mnl2[mc,:] = cumulated_regret_single_cb_mnl2[T-1]
  cumulated_regret_ucb_mnl[mc,:] = cumulated_regret_single_ucb_mnl[T-1]
  cumulated_regret_ts_mnl[mc,:] = cumulated_regret_single_ts_mnl[T-1]
  

  cumulated_regret_ucb_mnl_klow[mc,:] = cumulated_regret_single_ucb_mnl_klow[T-1]
  cumulated_regret_ts_mnl_klow[mc,:] = cumulated_regret_single_ts_mnl_klow[T-1]
  cumulated_regret_ucb_mnl_kinter[mc,:] = cumulated_regret_single_ucb_mnl_kinter[T-1]
  cumulated_regret_ts_mnl_kinter[mc,:] = cumulated_regret_single_ts_mnl_kinter[T-1]  
  


plottable_cumulated_regret_cb_mnl2 = np.mean(cumulated_regret_cb_mnl2,axis=0)
plottable_cumulated_regret_ucb_mnl = np.mean(cumulated_regret_ucb_mnl,axis=0)
plottable_cumulated_regret_ts_mnl = np.mean(cumulated_regret_ts_mnl,axis=0)

plottable_cumulated_regret_ucb_mnl_klow = np.mean(cumulated_regret_ucb_mnl_klow,axis=0)
plottable_cumulated_regret_ts_mnl_klow = np.mean(cumulated_regret_ts_mnl_klow,axis=0)

plottable_cumulated_regret_ucb_mnl_kinter = np.mean(cumulated_regret_ucb_mnl_kinter,axis=0)
plottable_cumulated_regret_ts_mnl_kinter = np.mean(cumulated_regret_ts_mnl_kinter,axis=0)


plottable_cumulated_regret_cb_mnl2_std = np.std(cumulated_regret_cb_mnl2,axis=0)
plottable_cumulated_regret_ucb_mnl_std = np.std(cumulated_regret_ucb_mnl,axis=0)
plottable_cumulated_regret_ts_mnl_std = np.std(cumulated_regret_ts_mnl,axis=0)

plottable_cumulated_regret_ucb_mnl_klow_std = np.std(cumulated_regret_ucb_mnl_klow,axis=0)
plottable_cumulated_regret_ts_mnl_klow_std = np.std(cumulated_regret_ts_mnl_klow,axis=0)

plottable_cumulated_regret_ucb_mnl_kinter_std = np.std(cumulated_regret_ucb_mnl_kinter,axis=0)
plottable_cumulated_regret_ts_mnl_kinter_std = np.std(cumulated_regret_ts_mnl_kinter,axis=0)


In [ ]:
import pickle
pickling_on = open("algo_comp_N_15_K_4_d_5_T_100_mc_25","wb")
pickle.dump(cumulated_regret_cb_mnl2, pickling_on) 
pickle.dump(cumulated_regret_ucb_mnl, pickling_on)
pickle.dump(cumulated_regret_ts_mnl, pickling_on)
pickle.dump(cumulated_regret_ucb_mnl_klow, pickling_on)
pickle.dump(cumulated_regret_ts_mnl_klow, pickling_on)
pickle.dump(cumulated_regret_ucb_mnl_kinter, pickling_on)
pickle.dump(cumulated_regret_ts_mnl_kinter, pickling_on)
pickling_on.close()

In [ ]:
random.seed(0)
np.random.seed(0)

N = 10
K = 6
d = 3
T = 100
dist = 0
#T0 = 5*np.sqrt(d)

sigma_sq=1.
rho_sq= 0
W=(sigma_sq-rho_sq)*np.eye(N) + rho_sq*np.ones((N,N))
MC_counts = 25
cumulated_regret_cb_mnl2 = np.zeros((MC_counts,T))
cumulated_regret_single_cb_mnl2 =[]
cumulated_regret_ucb_mnl = np.zeros((MC_counts,T))
cumulated_regret_single_ucb_mnl =[]
cumulated_regret_ts_mnl = np.zeros((MC_counts,T))
cumulated_regret_single_ts_mnl =[]
cumulated_regret_ucb_mnl_klow = np.zeros((MC_counts,T))
cumulated_regret_single_ucb_mnl_klow =[]
cumulated_regret_ts_mnl_klow = np.zeros((MC_counts,T))
cumulated_regret_single_ts_mnl_klow =[]

T0 = 5*np.sqrt(d)
for mc in range(MC_counts):
  cumulated_regret_single_cb_mnl2 =[]
  cumulated_regret_single_ucb_mnl =[]
  cumulated_regret_single_ts_mnl =[]
  cumulated_regret_single_ucb_mnl_klow =[]
  cumulated_regret_single_ts_mnl_klow =[]
  theta=np.random.uniform(0.,1.,d)
  env=mnlEnv(theta, K)
  env_theta= theta
  lbd=.01*d*np.log(K*T)
  M1=cb_mnl2(N=N, K=K, d=d, lbd=lbd, env_theta=env_theta)
  M2=ucb_mnl(N=N, K=K, d=d, T0=T0, kappa = .001)
  M3=ts_mnl(N=N, K=K, d=d, T0=T0, kappa = .001)
  M4=ucb_mnl(N=N, K=K, d=d, T0=T0, kappa = 1)
  M5=ts_mnl(N=N, K=K, d=d, T0=T0, kappa = 1)
  RWD1=list()
  optRWD=list()
  RWD2=list()
  optRWD2=list()
  RWD3=list()
  optRWD3=list()
  RWD4=list()
  optRWD4=list()
  RWD5=list()
  optRWD5=list()
  for t in range(T):
    lbd=.01*d*np.log(K*T)
    if dist == 0:
      x=np.random.multivariate_normal(np.zeros(N),W,d).T
    elif dist == 1:
      x=(np.random.random((N, d)) * 2 - 1)
    elif dist == 2:
      x=sample_elliptical(N, d, int(d/2), 0).T
	
    S1=M1.choose_S(t+1,x)
    rwd1, Y1 = env.compute_rwd(np.dot(x[S1,:],theta))
    RWD1.append(rwd1)
    M1.update_theta(Y1,t+1)
    opt_rwd = env.get_opt_rwd(x)
    optRWD.append(opt_rwd)
    cumulated_regret_single_cb_mnl2.append(np.cumsum(optRWD)-np.cumsum(RWD1))
    
    S2=M2.choose_S(t+1,x)
    rwd2, Y2 = env.compute_rwd(np.dot(x[S2,:],theta))
    RWD2.append(rwd2)
    M2.update_theta(Y2,t+1)
    opt_rwd2 = env.get_opt_rwd(x)
    optRWD2.append(opt_rwd2)
    cumulated_regret_single_ucb_mnl.append(np.cumsum(optRWD2)-np.cumsum(RWD2))

    S3=M3.choose_S(t+1,x)
    rwd3, Y3 = env.compute_rwd(np.dot(x[S3,:],theta))
    RWD3.append(rwd3)
    M3.update_theta(Y3,t+1)
    opt_rwd3 = env.get_opt_rwd(x)
    optRWD3.append(opt_rwd3)
    cumulated_regret_single_ts_mnl.append(np.cumsum(optRWD3)-np.cumsum(RWD3))
  
    S4=M4.choose_S(t+1,x)
    rwd4, Y4 = env.compute_rwd(np.dot(x[S4,:],theta))
    RWD4.append(rwd4)
    M4.update_theta(Y4,t+1)
    opt_rwd4 = env.get_opt_rwd(x)
    optRWD4.append(opt_rwd4)
    cumulated_regret_single_ucb_mnl_klow.append(np.cumsum(optRWD4)-np.cumsum(RWD4))
  
    S5=M5.choose_S(t+1,x)
    rwd5, Y5 = env.compute_rwd(np.dot(x[S5,:],theta))
    RWD5.append(rwd5)
    M5.update_theta(Y5,t+1)
    opt_rwd5 = env.get_opt_rwd(x)
    optRWD5.append(opt_rwd5)
    cumulated_regret_single_ts_mnl_klow.append(np.cumsum(optRWD5)-np.cumsum(RWD5))
  

  cumulated_regret_cb_mnl2[mc,:] = cumulated_regret_single_cb_mnl2[T-1]
  cumulated_regret_ucb_mnl[mc,:] = cumulated_regret_single_ucb_mnl[T-1]
  cumulated_regret_ts_mnl[mc,:] = cumulated_regret_single_ts_mnl[T-1]
  cumulated_regret_ucb_mnl_klow[mc,:] = cumulated_regret_single_ucb_mnl_klow[T-1]
  cumulated_regret_ts_mnl_klow[mc,:] = cumulated_regret_single_ts_mnl_klow[T-1]  


plottable_cumulated_regret_cb_mnl2 = np.mean(cumulated_regret_cb_mnl2,axis=0)
plottable_cumulated_regret_ucb_mnl = np.mean(cumulated_regret_ucb_mnl,axis=0)
plottable_cumulated_regret_ts_mnl = np.mean(cumulated_regret_ts_mnl,axis=0)

plottable_cumulated_regret_ucb_mnl_klow = np.mean(cumulated_regret_ucb_mnl_klow,axis=0)
plottable_cumulated_regret_ts_mnl_klow = np.mean(cumulated_regret_ts_mnl_klow,axis=0)


In [ ]:
pickling_on = open("algo_comp_N_10_K_6_d_3_T_100_mc_25","wb")
pickle.dump(cumulated_regret_cb_mnl2, pickling_on) 
pickle.dump(cumulated_regret_ucb_mnl, pickling_on)
pickle.dump(cumulated_regret_ts_mnl, pickling_on)
pickle.dump(cumulated_regret_ucb_mnl_klow, pickling_on)
pickle.dump(cumulated_regret_ts_mnl_klow, pickling_on)
pickling_on.close()

In [ ]:
random.seed(0)
np.random.seed(0)

N = 20
K = 5
d = 3
T = 100
dist = 0

sigma_sq=1.
rho_sq= 0
W=(sigma_sq-rho_sq)*np.eye(N) + rho_sq*np.ones((N,N))

MC_counts = 25
cumulated_regret_cb_mnl2 = np.zeros((MC_counts,T))

cumulated_regret_single_cb_mnl2 =[]



cumulated_regret_ucb_mnl = np.zeros((MC_counts,T))

cumulated_regret_single_ucb_mnl =[]



cumulated_regret_ts_mnl = np.zeros((MC_counts,T))

cumulated_regret_single_ts_mnl =[]



cumulated_regret_ucb_mnl_klow = np.zeros((MC_counts,T))

cumulated_regret_single_ucb_mnl_klow =[]



cumulated_regret_ts_mnl_klow = np.zeros((MC_counts,T))

cumulated_regret_single_ts_mnl_klow =[]



T0 = 5*np.sqrt(d)
for mc in range(MC_counts):
  cumulated_regret_single_cb_mnl2 =[]
  cumulated_regret_single_ucb_mnl =[]
  cumulated_regret_single_ts_mnl =[]
  cumulated_regret_single_ucb_mnl_klow =[]
  cumulated_regret_single_ts_mnl_klow =[]
  theta=np.random.uniform(0.,1.,d)
  env=mnlEnv(theta, K)
  env_theta= theta
  lbd=.01*d*np.log(K*T)
  M1=cb_mnl2(N=N, K=K, d=d, lbd=lbd, env_theta=env_theta)
  M2=ucb_mnl(N=N, K=K, d=d, T0=T0, kappa = .001)
  M3=ts_mnl(N=N, K=K, d=d, T0=T0, kappa = .001)
  M4=ucb_mnl(N=N, K=K, d=d, T0=T0, kappa = 1)
  M5=ts_mnl(N=N, K=K, d=d, T0=T0, kappa = 1)
  RWD1=list()
  optRWD=list()
  RWD2=list()
  optRWD2=list()
  RWD3=list()
  optRWD3=list()

  RWD4=list()
  optRWD4=list()
  RWD5=list()
  optRWD5=list()
  for t in range(T):
    lbd=.01*d*np.log(K*T)
    if dist == 0:
      x=np.random.multivariate_normal(np.zeros(N),W,d).T
    elif dist == 1:
      x=(np.random.random((N, d)) * 2 - 1)
    elif dist == 2:
      x=sample_elliptical(N, d, int(d/2), 0).T
	
    S1=M1.choose_S(t+1,x)
    rwd1, Y1 = env.compute_rwd(np.dot(x[S1,:],theta))
    RWD1.append(rwd1)
    M1.update_theta(Y1,t+1)
    opt_rwd = env.get_opt_rwd(x)
    optRWD.append(opt_rwd)
    cumulated_regret_single_cb_mnl2.append(np.cumsum(optRWD)-np.cumsum(RWD1))
    
    S2=M2.choose_S(t+1,x)
    rwd2, Y2 = env.compute_rwd(np.dot(x[S2,:],theta))
    RWD2.append(rwd2)
    M2.update_theta(Y2,t+1)
    opt_rwd2 = env.get_opt_rwd(x)
    optRWD2.append(opt_rwd2)
    cumulated_regret_single_ucb_mnl.append(np.cumsum(optRWD2)-np.cumsum(RWD2))

    S3=M3.choose_S(t+1,x)
    rwd3, Y3 = env.compute_rwd(np.dot(x[S3,:],theta))
    RWD3.append(rwd3)
    M3.update_theta(Y3,t+1)
    opt_rwd3 = env.get_opt_rwd(x)
    optRWD3.append(opt_rwd3)
    cumulated_regret_single_ts_mnl.append(np.cumsum(optRWD3)-np.cumsum(RWD3))
  
    S4=M4.choose_S(t+1,x)
    rwd4, Y4 = env.compute_rwd(np.dot(x[S4,:],theta))
    RWD4.append(rwd4)
    M4.update_theta(Y4,t+1)
    opt_rwd4 = env.get_opt_rwd(x)
    optRWD4.append(opt_rwd4)
    cumulated_regret_single_ucb_mnl_klow.append(np.cumsum(optRWD4)-np.cumsum(RWD4))
  
    S5=M5.choose_S(t+1,x)
    rwd5, Y5 = env.compute_rwd(np.dot(x[S5,:],theta))
    RWD5.append(rwd5)
    M5.update_theta(Y5,t+1)
    opt_rwd5 = env.get_opt_rwd(x)
    optRWD5.append(opt_rwd5)
    cumulated_regret_single_ts_mnl_klow.append(np.cumsum(optRWD5)-np.cumsum(RWD5))
  

  cumulated_regret_cb_mnl2[mc,:] = cumulated_regret_single_cb_mnl2[T-1]
  cumulated_regret_ucb_mnl[mc,:] = cumulated_regret_single_ucb_mnl[T-1]
  cumulated_regret_ts_mnl[mc,:] = cumulated_regret_single_ts_mnl[T-1]
  cumulated_regret_ucb_mnl_klow[mc,:] = cumulated_regret_single_ucb_mnl_klow[T-1]
  cumulated_regret_ts_mnl_klow[mc,:] = cumulated_regret_single_ts_mnl_klow[T-1]  


plottable_cumulated_regret_cb_mnl2 = np.mean(cumulated_regret_cb_mnl2,axis=0)
plottable_cumulated_regret_ucb_mnl = np.mean(cumulated_regret_ucb_mnl,axis=0)
plottable_cumulated_regret_ts_mnl = np.mean(cumulated_regret_ts_mnl,axis=0)

plottable_cumulated_regret_ucb_mnl_klow = np.mean(cumulated_regret_ucb_mnl_klow,axis=0)
plottable_cumulated_regret_ts_mnl_klow = np.mean(cumulated_regret_ts_mnl_klow,axis=0)



In [ ]:
pickling_on = open("algo_comp_N_20_K_5_d_3_T_100_mc_25","wb")
pickle.dump(cumulated_regret_cb_mnl2, pickling_on) 
pickle.dump(cumulated_regret_ucb_mnl, pickling_on)
pickle.dump(cumulated_regret_ts_mnl, pickling_on)
pickle.dump(cumulated_regret_ucb_mnl_klow, pickling_on)
pickle.dump(cumulated_regret_ts_mnl_klow, pickling_on)
pickling_on.close()